Needed somewhere to test model summary

In [1]:
%run setup.ipynb

No of images in training set: 835
No of images in validation set: 104
No of images in test set: 105
No of images in feeder set: 15079


In [28]:
model = load_and_prep_resnet50()
model #print model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
model.eval()
truel, predl = labels_and_predictions(dataloader_validation, model, device)

In [23]:
acc = accuracy(truel, predl)
print("Accuracy ", acc)
print(classification_report(truel, predl, target_names=class_labels))

Accuracy  10.576923076923077
              precision    recall  f1-score   support

     blueTit       0.00      0.00      0.00        15
   chaffinch       0.17      0.07      0.10        15
     coalTit       0.19      0.50      0.28        14
   goldfinch       0.00      0.00      0.00        15
    greatTit       0.00      0.00      0.00        15
       robin       0.05      0.07      0.06        15
    starling       0.33      0.13      0.19        15

    accuracy                           0.11       104
   macro avg       0.11      0.11      0.09       104
weighted avg       0.11      0.11      0.09       104



In [ ]:
model_frozen_status(model)

In [ ]:
unfreeze_layers(model)
freeze_below_4(model)
model_frozen_status(model) # now layer 4 is unfrozen

In [ ]:
model = load_and_prep_resnet50() #all frozen
unfreeze_layer4(model)
model_frozen_status(model) # now layer 4 is unfrozen

In [ ]:
unfreeze_layer3(model)
model_frozen_status(model) #now layer 3 is also unfrozen

In [45]:
def load_and_prep_vgg16():
    model = torchvision.models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=7)

    #Freeze layers
    for param in model.features.parameters():
      param.requires_grad = False

    #Unfreeze classifier
    for param in model.classifier.parameters():
      param.requires_grad = True

    model = model.to(device)

    return model

In [43]:
model_vgg = load_and_prep_vgg16()

In [8]:
truel_vgg, predl_vgg = labels_and_predictions(dataloader_test, model_vgg, device)

In [ ]:
vgg_acc = accuracy(truel_vgg, predl_vgg)
print("Accuracy ", vgg_acc)

Accuracy  11.428571428571429


In [24]:
print(classification_report(truel_vgg, predl_vgg, target_names=class_labels))

              precision    recall  f1-score   support

     blueTit       0.00      0.00      0.00        15
   chaffinch       0.17      0.27      0.21        15
     coalTit       0.11      0.07      0.08        15
   goldfinch       0.00      0.00      0.00        15
    greatTit       0.15      0.33      0.21        15
       robin       0.00      0.00      0.00        15
    starling       0.14      0.13      0.14        15

    accuracy                           0.11       105
   macro avg       0.08      0.11      0.09       105
weighted avg       0.08      0.11      0.09       105



In [25]:
model_vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [41]:
total_params = sum(p.numel() for p in model_vgg.parameters())
print(f'{total_params:,} total parameters in VGG16.')
total_trainable_params = sum(
    p.numel() for p in model_vgg.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

134,289,223 total parameters in VGG16.
119,574,535 training parameters.


In [46]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters in ResNet50.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

23,522,375 total parameters in ResNet50.
14,343 training parameters.


In [47]:
unfreeze_layer4(model)
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters in ResNet50.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

23,522,375 total parameters in ResNet50.
14,979,079 training parameters.


In [52]:
from pytorch_model_summary import summary
unfreeze_layer3(model)
print(summary(model, torch.zeros((1,3,224,224))))

------------------------------------------------------------------------------
           Layer (type)          Output Shape         Param #     Tr. Param #
               Conv2d-1     [1, 64, 112, 112]           9,408               0
          BatchNorm2d-2     [1, 64, 112, 112]             128               0
                 ReLU-3     [1, 64, 112, 112]               0               0
            MaxPool2d-4       [1, 64, 56, 56]               0               0
           Bottleneck-5      [1, 256, 56, 56]          75,008               0
           Bottleneck-6      [1, 256, 56, 56]          70,400               0
           Bottleneck-7      [1, 256, 56, 56]          70,400               0
           Bottleneck-8      [1, 512, 28, 28]         379,392               0
           Bottleneck-9      [1, 512, 28, 28]         280,064               0
          Bottleneck-10      [1, 512, 28, 28]         280,064               0
          Bottleneck-11      [1, 512, 28, 28]         280,064  

In [51]:
from pytorch_model_summary import summary

print(summary(model_vgg, torch.zeros((1,3,224,224))))

-------------------------------------------------------------------------------
           Layer (type)           Output Shape         Param #     Tr. Param #
               Conv2d-1      [1, 64, 224, 224]           1,792               0
                 ReLU-2      [1, 64, 224, 224]               0               0
               Conv2d-3      [1, 64, 224, 224]          36,928               0
                 ReLU-4      [1, 64, 224, 224]               0               0
            MaxPool2d-5      [1, 64, 112, 112]               0               0
               Conv2d-6     [1, 128, 112, 112]          73,856               0
                 ReLU-7     [1, 128, 112, 112]               0               0
               Conv2d-8     [1, 128, 112, 112]         147,584               0
                 ReLU-9     [1, 128, 112, 112]               0               0
           MaxPool2d-10       [1, 128, 56, 56]               0               0
              Conv2d-11       [1, 256, 56, 56]     